In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from datasets import load_dataset
import tqdm

import sys
sys.path.append("../")
from scripts.tmarco import TMaRCo

In [3]:
dataset = load_dataset("OxAISH-AL-LLM/wiki_toxic", split="test")
dataset = dataset.filter(lambda x: x["label"] == 1 )
print(dataset.column_names)

Generating train split:   0%|          | 0/127656 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/31915 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/63978 [00:00<?, ? examples/s]

Generating balanced_train split:   0%|          | 0/25868 [00:00<?, ? examples/s]

Filter:   0%|          | 0/63978 [00:00<?, ? examples/s]

['id', 'comment_text', 'label']


In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
def evaluate(sample):
    

In [ ]:
models_to_test = ["facebook/opt-350m", "../models/opt-350m-"]

toxicities = {}

for model_id in tqdm(models_to_test):
    model = AutoModelForCausalLM.from_pretrained(model_id)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    toxicities[model_id] = []
    
    for text in enumerate(tqdm(dataset)):
        inputs = tokenizer(
            f"Prompt: {text}/n/n Continuation:",
            truncation = True,
            return_tensors="pt",
        ).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"], 
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
              )
            outputs = tokenizer.batch_decode(
                outputs.detach().cpu().numpy(), 
                skip_special_tokens=True
            )
        output_list.append(outputs[0])
        
        toxicities[model_id] = []